In [1]:
import random
import numpy as np
import crocoddyl
import keras
from keras.models import load_model
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
crocoddyl.switchToNumpyMatrix()


Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
def iteration_analysis(MAXITER:int =1000):
    """
    Script to store the number of iterations crocoddyl takes to generate a trajectory.
    Here, we generate 10K trajectories and store the number of iterations taken by  crocoddyl 
    for each trajectory
    
    """
    iterations = []
    not_feasible = 0
    
    NUM_TRAJ = 10000


    for _ in range(NUM_TRAJ):
        initial_config = [random.uniform(-2.1, 2.),
                                    random.uniform(-2.1, 2.), 
                                    random.uniform(0, 1)]

        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1, 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)

        ddp = crocoddyl.SolverDDP(problem)

        done = ddp.solve([], [], MAXITER)
        if not ddp.isFeasible:
            not_feasible += 1

        else:
            iterations.append(ddp.iter)

    max_value = max(iterations)
    min_value = min(iterations)
    avg_value = sum(iterations)//len(iterations)
    print(f"Total trajectories: {NUM_TRAJ}, not feasible trajectories: {not_feasible}\n\n")
    print(f'Maximum iterations allowed : {MAXITER}\n')
    print(f"\nMinimum iterations taken -> {min_value},\
          Maximum iterations taken -> {max_value},\
          Average -> {avg_value} \n")
    print(".............................\n")
    
    return iterations

Group1 = iteration_analysis(MAXITER = 1000)
Group2 = iteration_analysis(MAXITER = 100)
Group3 = iteration_analysis(MAXITER = 500)

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 1000


Minimum iterations taken -> 2,          Maximum iterations taken -> 1000,          Average -> 26 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 100


Minimum iterations taken -> 2,          Maximum iterations taken -> 100,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 500


Minimum iterations taken -> 2,          Maximum iterations taken -> 500,          Average -> 26 

.............................



In [4]:
from data_net import *
import random
import numpy as np
import crocoddyl
from keras.models import load_model

def warmstart(n_trajectories: int = 10000, load_net: bool = False):
    if load_net:
        neuralNet = load_model('basic_model.h5')
    else:
        neuralNet = train_net()
        
    print(f" With warmstart, without warmstart ")
    
    for _ in range(n_trajectories):
            
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]

            model = crocoddyl.ActionModelUnicycle()
            model2 = crocoddyl.ActionModelUnicycle()
            
            model.costWeights = np.matrix([1., .3]).T
            model2.costWeights = np.matrix([1., .3]).T
    
            
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
            problem2 = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model2 ] * 20, model2)

            
            ddp = crocoddyl.SolverDDP(problem)
            ddp2 = crocoddyl.SolverDDP(problem2)
            
            
            x_data = np.matrix(initial_config).reshape(1, 3)

            prediction = neuralNet.predict(x_data).reshape(20, 5)

            xs = np.matrix(prediction[:, 0:3])

            us = np.matrix(prediction[:, 3:5])
            

            ddp_xs = []
            ddp_xs.append(np.matrix(initial_config).reshape(1, 3).T)
            for _ in  range(xs.shape[0]):
                ddp_xs.append((np.matrix(xs[_]).T))

            ddp_us = []
            for _ in  range(us.shape[0]):
                ddp_us.append(np.matrix(us[_]).T)

            ddp.solve(ddp_xs,ddp_us, 100)
            ddp2.solve()
            print("  ", ddp.iter, "  ", ddp2.iter)
                
warmstart(n_trajectories=1000, load_net=True)    

 With warmstart, without warmstart 
   13    12
   13    15
   30    28
   48    42
   18    19
   18    18
   78    84
   100    38
   3    3
   12    10
   18    23
   9    8
   7    7
   6    5
   23    24
   20    19
   11    10
   13    15
   65    37
   53    71
   29    28
   51    65
   21    22
   67    67
   33    38
   15    16
   37    39
   21    20
   63    32
   17    15
   27    27
   13    13
   16    15
   66    24
   6    5
   29    32
   4    3
   5    4
   6    7
   5    4
   51    47
   24    24
   20    21
   6    6
   40    31
   31    30
   22    18
   7    7
   46    44
   100    100
   94    85
   20    18
   4    4
   33    26
   45    54
   32    36
   100    68
   18    15
   81    81
   37    48
   21    20
   9    11
   4    4
   25    24
   31    35
   30    33
   14    13
   19    24
   22    29
   27    25
   23    22
   27    26
   26    30
   16    17
   6    5
   10    10
   100    47
   19    17
   29    34
   13    13
   100    56
   27    25
   

   58    51
   11    9
   10    9
   37    29
   12    13
   36    30
   16    16
   17    16
   15    14
   18    18
   45    46
   20    17
   6    6
   8    8
   7    8
   32    33
   30    29
   21    19
   9    7
   31    35
   8    9
   23    22
   20    20
   28    27
   4    4
   86    62
   7    8
   94    60
   43    59
   32    29
   34    30
   5    5
   69    60
   5    5
   5    5
   4    5
   25    27
   100    48
   7    7
   15    17
   72    100
   33    38
   44    45
   6    6
   16    14
   12    12
   100    64
   16    18
   8    7
   39    30
   24    23
   58    64
   23    22
   5    5
   39    31
   24    23
   6    6
   11    11
   23    22
   82    74
   42    38
   26    29
   18    19
   10    12
   21    21
   64    57
   45    41
   5    5
   9    9
   40    33
   6    5
   17    15
   22    26
   17    23
   85    100
   34    31
   9    9
   59    76
   33    34
   6    6
   24    22
   35    34
   35    36
   21    18
   51    57
   19    18
   8    

In [ ]:
#TO DO